In [0]:
from pyspark.sql.functions import col, count, wh

---------------------------------------------------------------------------
ImportError                               Traceback (most recent call last)
File <command-46374841226772>:1
----> 1 from pyspark.sql.functions import col, count, wh

ImportError: cannot import name 'wh' from 'pyspark.sql.functions' (/databricks/spark/python/pyspark/sql/functions.py)

In [0]:
# Leitura das tabelas Bronze e Silver
df_orders = spark.read.table("bronze.olist_orders")
df_order_items = spark.read.table("bronze.olist_order_items")
df_dim_customers = spark.read.table("silver.dim_customers")

### JOIN

In [0]:
# Join: order_items, orders e customers
df_fato = df_order_items \
    .join(df_orders, on="order_id", how="inner") \
    .join(df_dim_customers, on="customer_id", how="inner")

### Preço total item

In [0]:
# Seleção sugerida no desafio com coluna com métrica sugerida de preço total do item. 
df_fato_final = df_fato.select(
    "order_id",
    "order_item_id",
    "product_id",
    "seller_id",
    "customer_id",
    "order_status",
    "order_purchase_timestamp",
    "shipping_limit_date",
    "price",
    "freight_value"
).withColumn("preco_total_item", col("price") + col("freight_value"))


### Preço médio unitário

In [0]:
# Calcula o preço médio por item no pedido, preco_unitario_medio = preco_total_item / order_item_id

df_fato_final = df_fato_final.withColumn(
    "preco_medio_unitario", col("preco_total_item") / col("order_item_id")
)

### Verificações

#### Campos Nulos

In [0]:
# Verificação de campos nulos em colunas essenciais
null_check = df_fato_final.select([
    count(when(col(c).isNull(), c)).alias(c) for c in [
        "order_id", "order_item_id", "product_id", "customer_id",
        "price", "freight_value", "preco_total_item"
    ]
])

display(null_check)

# Se houver nulos, interrompe o notebook
null_counts = null_check.collect()[0].asDict()
if any(v > 0 for v in null_counts.values()):
    raise Exception(f"Falha na verificação de qualidade: campos nulos encontrados: {null_counts}")


order_id,order_item_id,product_id,customer_id,price,freight_value,preco_total_item
0,0,0,0,0,0,0


#### Valores Negativos

In [0]:
# Verifica se há preços negativos
valores_invalidos = df_fato_final.filter((col("price") < 0) | (col("freight_value") < 0))

display(valores_invalidos)

if valores_invalidos.count() > 0:
    raise Exception("Falha na verificação de qualidade: valores negativos encontrados em price ou freight_value.")

order_id,order_item_id,product_id,seller_id,customer_id,order_status,order_purchase_timestamp,shipping_limit_date,price,freight_value,preco_total_item,preco_medio_unitario


In [0]:
df_fato_final.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver.fct_order_items")